In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("..")

In [4]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [6]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [17]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [15]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")
parsed_rom = rom.parse()

In [18]:
all_posns = all_global_positions(rooms, parsed_rom)

Landing_Site 0x791f8
	 Extra: []
	 Missing: ['Landing_Site_Ship']
West_Ocean 0x793fe
	 Extra: ['West_Ocean_L', 'West_Ocean_R4']
	 Missing: ['West_Ocean_R6', 'West_Ocean_L2']
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: ['Bomb_Torizo_Bombs']
Spore_Spawn 0x79dc7
	 Extra: []
	 Missing: ['Spore_Spawn_Spore_Spawn']
Kraid 0x7a59f
	 Extra: []
	 Missing: ['Kraid_Kraid']
Statues 0x7a66a
	 Extra: []
	 Missing: ['Statues_Statues']
Crocomire 0x7a98d
	 Extra: []
	 Missing: ['Crocomire_Crocomire']
Water_Closet 0x7b1e5
	 Extra: []
	 Missing: ['Water_Closet_Drain']
Golden_Torizo 0x7b283
	 Extra: []
	 Missing: ['Golden_Torizo_Golden_Torizo']
Ridley 0x7b32e
	 Extra: []
	 Missing: ['Ridley_Ridley']
Phantoon 0x7cd13
	 Extra: []
	 Missing: ['Phantoon_Phantoon']
West_Sand_Hall 0x7d461
	 Extra: []
	 Missing: ['West_Sand_Hall_TS']
East_Sand_Hall 0x7d4c2
	 Extra: []
	 Missing: ['East_Sand_Hall_TS']
West_Sand_Hole 0x7d4ef
	 Extra: []
	 Missing: ['West_Sand_Hole_TS']
East_Sand_Hole 0x7d51e
	 Extra: 

In [11]:
import omega
from omega.symbolic.fol import Context

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [20]:
#all_posns

In [118]:
design["Rooms"]["Botwoon_Energy"]

{'Nodes': ['L', 'R', 'E', 'BS1', 'BS2'],
 'Drops': {'E': 'E'},
 'Edges': {'L': [{'Terminal': 'R',
    'Requirements': [['GS', 'SB'],
     ['B', 'GS', 'MB'],
     ['PB', 'GS', 'MB'],
     ['SPB', 'GS', 'MB'],
     ['SPB', 'HJ', 'MB']]},
   {'Terminal': 'E', 'Requirements': [['GS', 'MB'], ['HJ', 'MB']]},
   {'Terminal': 'BS1', 'Requirements': [[]]},
   {'Terminal': 'BS2', 'Requirements': [[]]}],
  'R': [{'Terminal': 'L',
    'Requirements': [['B', 'GS', 'MB'],
     ['PB', 'GS', 'MB'],
     ['SPB', 'HJ', 'MB'],
     ['SPB', 'GS', 'MB']]},
   {'Terminal': 'E',
    'Requirements': [['B', 'GS', 'MB'],
     ['PB', 'GS', 'MB'],
     ['SPB', 'HJ', 'MB'],
     ['SPB', 'GS', 'MB']]},
   {'Terminal': 'BS1',
    'Requirements': [['B', 'GS', 'MB'],
     ['PB', 'GS', 'MB'],
     ['SPB', 'HJ', 'MB'],
     ['SPB', 'GS', 'MB']]},
   {'Terminal': 'BS2',
    'Requirements': [['B', 'GS', 'MB'],
     ['PB', 'GS', 'MB'],
     ['SPB', 'HJ', 'MB'],
     ['SPB', 'GS', 'MB']]}],
  'E': [{'Terminal': 'L', 'Requir

In [30]:
all_global_positions

<function abstraction_validation.sm_paths.all_global_positions(rooms, parsed_rom)>

In [33]:
design["Items"], design["Bosses"]

({'B': 'Bombs',
  'PB': 'Power_Bombs',
  'SPB': 'Spring_Ball',
  'S': 'Super_Missiles',
  'M': 'Missiles',
  'G': 'Grapple_Beam',
  'SA': 'Screw_Attack',
  'V': 'Varia_Suit',
  'GS': 'Gravity_Suit',
  'SB': 'Speed_Booster',
  'HJ': 'Hi_Jump',
  'MB': 'Morph',
  'CB': 'Charge_Beam',
  'WB': 'Wave_Beam',
  'E': 'Energy_Tank',
  'PLB': 'Plasma_Beam',
  'IB': 'Ice_Beam',
  'SJ': 'Space_Jump',
  'Spazer': 'Spazer',
  'RT': 'Reserve_Tank',
  'XR': 'XRay',
  'Drain': 'Drain',
  'Shaktool': 'Shaktool',
  'START': 'START',
  'Statues': 'Statues'},
 {'Kraid': 'Kraid',
  'Phantoon': 'Phantoon',
  'Draygon': 'Draygon',
  'Ridley': 'Ridley',
  'Mother_Brain': 'Mother_Brain',
  'Botwoon': 'Botwoon',
  'Spore_Spawn': 'Spore_Spawn',
  'Golden_Torizo': 'Golden_Torizo',
  'Bomb_Torizo': 'Bomb_Torizo',
  'Crocomire': 'Crocomire'})

In [122]:
def loc(room_name, node_name, when="prev"):
    #TODO: local positions
    #TODO: with room_prev and room_next
    try:
        node_loc = all_posns[f"{room_name}_{node_name}"]
    except KeyError:
        return "FALSE"
    xx = f"x_{when} = {int(node_loc.x)}"
    yy = f"y_{when} = {int(node_loc.y)}"
    #TODO room_id
    return " & ".join([xx, yy])

def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)

def required_itemsets(itemsets):
    return "(" + " | ".join([" & ".join([f"{item}_prev = 1" for item in itemset]) for itemset in itemsets]) + ")"


In [102]:
from tqdm.notebook import tqdm

In [125]:
items_unchanged_def = "items_unchanged == " + " & ".join([f"{i}_prev = {i}_next" for i in design["Items"] | design["Bosses"]])

In [126]:
items_unchanged_def

'items_unchanged == B_prev = B_next & PB_prev = PB_next & SPB_prev = SPB_next & S_prev = S_next & M_prev = M_next & G_prev = G_next & SA_prev = SA_next & V_prev = V_next & GS_prev = GS_next & SB_prev = SB_next & HJ_prev = HJ_next & MB_prev = MB_next & CB_prev = CB_next & WB_prev = WB_next & E_prev = E_next & PLB_prev = PLB_next & IB_prev = IB_next & SJ_prev = SJ_next & Spazer_prev = Spazer_next & RT_prev = RT_next & XR_prev = XR_next & Drain_prev = Drain_next & Shaktool_prev = Shaktool_next & START_prev = START_next & Statues_prev = Statues_next & Kraid_prev = Kraid_next & Phantoon_prev = Phantoon_next & Draygon_prev = Draygon_next & Ridley_prev = Ridley_next & Mother_Brain_prev = Mother_Brain_next & Botwoon_prev = Botwoon_next & Spore_Spawn_prev = Spore_Spawn_next & Golden_Torizo_prev = Golden_Torizo_next & Bomb_Torizo_prev = Bomb_Torizo_next & Crocomire_prev = Crocomire_next'

In [129]:
doors = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        links.append(loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions())
        if node_name in room['Drops']:
            links.append(loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]))
    for node_name, door in room['Doors'].items():
        doors.append(loc(room_name, node_name) + " & " + loc(door['Room'], door['Node'], when="next") + " & " + item_transitions())
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            links.append(loc(room_name, node_name) + " & " + loc(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions())
    link_str = " | ".join(["(" + l + ")" for l in links])
    fname = f"../output/tla/{room_name}.tla"
    with open(fname, "w") as f:
        f.write(f"LET {items_unchanged_def} IN {link_str}")
door_fname = f"../output/tla/doors.tla"
door_str = " | ".join(doors)
with open(door_fname, "w") as f:
    f.write(door_str)

  0%|          | 0/255 [00:00<?, ?it/s]

In [130]:
from omega.logic.lexyacc import Parser

In [131]:
p = Parser()

In [132]:
%%time
with open("../output/tla/doors.tla") as f:
    s = f.read()
    p.parse(s)

CPU times: user 63.7 ms, sys: 0 ns, total: 63.7 ms
Wall time: 66.1 ms


In [128]:
p.parse("LET x == 7 IN y")

Operator('LET', [Binary('==', Terminal('x', 'opname'), Num('7', 'num'))], Var('y', 'var'))